In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import re

In [4]:
df_land = pd.read_csv("Land_Improvements.csv")
df_property = pd.read_csv("property_features.csv")
df_location = pd.read_csv("location.csv")

In [5]:
df_land.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Unnamed: 0    1000000 non-null  int64  
 1   cat3_slug     999999 non-null   object 
 2   Land          990897 non-null   float64
 3   Improvements  1000000 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 30.5+ MB


In [6]:
df_property.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Unnamed: 0     1000000 non-null  int64  
 1   cat3_slug      999999 non-null   object 
 2   has_parking    1000000 non-null  float64
 3   diff_age       1000000 non-null  float64
 4   has_warehouse  1000000 non-null  float64
 5   has_elevator   999911 non-null   float64
 6   rooms_count    1000000 non-null  float64
 7   floor          1000000 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 61.0+ MB


In [7]:
df_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   Unnamed: 0         1000000 non-null  int64 
 1   cat3_slug          999999 non-null   object
 2   city_slug          999998 non-null   object
 3   neighborhood_slug  999998 non-null   object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


In [ ]:
df_merged = df_location.merge(df_land.drop(columns=["cat3_slug"], errors='ignore'), on="Unnamed: 0", how="left")
df_merged = df_merged.merge(df_property.drop(columns=["cat3_slug"], errors='ignore'), on="Unnamed: 0", how="left")

cols_order = ["Unnamed: 0", "cat3_slug", "city_slug", "neighborhood_slug", "Land", "Improvements",
              "has_parking", "diff_age", "has_warehouse", "has_elevator", "rooms_count", "floor"]

df_merged = df_merged[cols_order]
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Unnamed: 0         1000000 non-null  int64  
 1   cat3_slug          999999 non-null   object 
 2   city_slug          999998 non-null   object 
 3   neighborhood_slug  999998 non-null   object 
 4   Land               990897 non-null   float64
 5   Improvements       1000000 non-null  float64
 6   has_parking        1000000 non-null  float64
 7   diff_age           1000000 non-null  float64
 8   has_warehouse      1000000 non-null  float64
 9   has_elevator       999911 non-null   float64
 10  rooms_count        1000000 non-null  float64
 11  floor              1000000 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 91.6+ MB


### Outlier handling

In [ ]:
df_subset.Land = np.log(df_subset.Land)
for clmn in df_subset.cat3_slug.unique():
    mask = (df_subset.cat3_slug == clmn)
    Q3 = df_subset.loc[mask, "Land"].quantile(0.75)
    Q1 = df_subset.loc[mask, "Land"].quantile(0.25)
    IQR_n = Q3 - Q1
    tr_max = Q3 + 1.5 * IQR_n
    tr_min = Q1 - 1.5 * IQR_n
    df_subset.loc[mask & (df_subset["Land"] > tr_max), "Land"] = tr_max
    df_subset.loc[mask & (df_subset["Land"] < tr_min), "Land"] = tr_min
    
df_subset.Land = np.exp(df_subset.Land)

In [ ]:
land_data_3 = land_data[["Unnamed: 0", "cat3_slug", "Land", "Improvements"]].copy()

# ensure Land is numeric and handle zeros/negatives
land_data["Land"] = pd.to_numeric(land_data["Land"], errors="coerce")
# Use log for positive values; non-positive -> NaN
land_data["Land_log"] = np.where(land_data["Land"] > 0, np.log(land_data["Land"]), np.nan)

cleaned_frames = []
min_count = 10   # don't attempt removal on groups smaller than this

for clmn in land_data["cat3_slug"].dropna().unique():
    grp = land_data[land_data["cat3_slug"] == clmn].copy()
    # consider only rows with valid log-values for IQR calculation
    valid = grp["Land_log"].dropna()
    if len(valid) < min_count:
        # skip removal for small groups (keep all rows as-is)
        cleaned_frames.append(grp)
        continue

    Q1 = valid.quantile(0.25)
    Q3 = valid.quantile(0.75)
    IQR_n = Q3 - Q1
    lower = Q1 - 1.5 * IQR_n
    upper = Q3 + 1.5 * IQR_n

    # keep only rows within bounds (on log scale)
    grp_filtered = grp[grp["Land_log"].between(lower, upper)]
    cleaned_frames.append(grp_filtered)

# combine back and preserve original index order
land_data_cleaned = pd.concat(cleaned_frames).sort_index()

# inverse-transform back to original scale
land_data_cleaned.loc[:, "Land"] = np.exp(land_data_cleaned["Land_log"])

# drop helper column
land_data_cleaned = land_data_cleaned.drop(columns=["Land_log"])

# land_data_cleaned now contains the rows kept after outlier removal
print("Original rows:", len(land_data), "Kept rows:", len(land_data_cleaned))

In [ ]:
train_df[train_df.cat3_slug == "partnership"].info()